# Heart Failure Precition - Projekt Podstawy Uczenia Maszynowego Laboratorum

In [ ]:
import numpy as np
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import LabelEncoder

Wczytujemy nasz zbór danych, jak możemy wywnioskować zbiór składa się z danych odnoszących się do 299 pacjentów. Posiadamy 12 różnych kolumn z informacjami na temat stanu zdrowia pacjenta oraz kolumnę `DEATH_EVENT` binarnie opisującą czy pacjent zmarł czy nie. (1 - jeżeli zmarł)

In [ ]:
dataset = pd.read_csv("../input/heart-failure-clinical-data/heart_failure_clinical_records_dataset.csv")
print(np.shape(dataset))
dataset

Skupmy się na chwilę na kolumnie `DEATH_EVENT` i sprawdźmy ile pacjentów zawartych w tabeli zmarło.

In [ ]:
print(dataset['DEATH_EVENT'].value_counts())
dataset['DEATH_EVENT'].value_counts().plot.pie(explode=[0,0.1],shadow=True,autopct='%1.1f%%',
                                              title = 'Is patient died')

Jak możemy zauważyć poniżej nasze dane tabelaryczne są pełne i nie zawierają żadnych pustych wpisów co ułatwia nam pracę.

In [ ]:
np.where(pd.isnull(dataset))

Wszystkie pomiary to wartości liczbowe, dzięki temu nie musimy kodować zmiennych kategorycznych.

In [ ]:
dataset.dtypes.value_counts()

Teraz sprawdzimy czy wszystkie dane wyglądają na prawidłowe. Użyjemy do tego metody `describe()`. Na pierwszy rzut oka żadne skrajne wartości w kolumnach nie odbiegają znacząco od normy, ale warto przyjrzeć się im bliżej przy wizualizacji.

In [ ]:
dataset.describe()

In [ ]:
dataset['age'].plot.hist(title = "Histogram of column AGE")
plt.xlabel('Age')

In [ ]:
dataset['creatinine_phosphokinase'].plot.hist(title = "Histogram of column creatinine_phosphokinase")
plt.xlabel('Level of the CPK enzyme in the blood (mcg/L)')

In [ ]:
 dataset['diabetes'].value_counts().plot.pie(explode=[0,0.1],shadow=True,autopct='%1.1f%%',
                                       title = 'Is patiernt diabetic')

In [ ]:
dataset['ejection_fraction'].plot.hist(title = "Histogram of column EJECTION_FRACTION")
plt.xlabel('Percentage of blood leaving the heart at each heart contraction')

In [ ]:
 dataset['high_blood_pressure'].value_counts().plot.pie(explode=[0,0.1],shadow=True,autopct='%1.1f%%',
                                       title = 'Is patiernt having hypertention')

In [ ]:
dataset['platelets'].plot.hist(title = "Histogram of column PLATELETS")
plt.xlabel('Platelets in the blood (kiloplatelets/mL)')

In [ ]:
dataset['serum_creatinine'].plot.hist(title = "Histogram of column SERUM_CREATININE")
plt.xlabel('Level of serum creatinine in the blood (mg/dL)')

In [ ]:
dataset['serum_sodium'].plot.hist(title = "Histogram of column SERUM_SODIUM")
plt.xlabel('Level of serum sodium in the blood (mEq/L)')

In [ ]:
 dataset['sex'].value_counts().plot.pie(explode=[0,0.1],shadow=True,autopct='%1.1f%%',
                                       title= 'Is patient male')

In [ ]:
 dataset['smoking'].value_counts().plot.pie(explode=[0,0.1],shadow=True,autopct='%1.1f%%',
                                       title = 'Does patient smokes')

In [ ]:
dataset['time'].plot.hist(title = "Histogram of column TIME")
plt.xlabel('Follow-up period (days)')

Gdy zapoznaliśmy się już dokładniej z danymi, warto byłoby utworzyć macierz korelacji pomiędzy nimi. Z utworzonej poniżej macierzy widzimy że cztery pomiary które mają największą korelacje z wartością kolumny `DEATH_EVENT` to
* `time`(-0.53) oraz `ejection_fraction`(-0,27) - Ujemna korelacja
* `serum_creatinine` (0.29) oraz `age` (0.25) - Dodatnia korelacja

In [ ]:
corr = dataset.corr()
f = plt.figure(figsize = (14,9))
sns.heatmap(corr,vmax = 1,square = True,annot = True,vmin = -1)
plt.show()

Po ogólnej analizie naszego zbioru danych przechodzimy do porównania cech które najbardziej wpływają na śmierć z kolumną `DEATH_EVENT`

Jak można zauważyć na poniższym wykresie dla kreatyniny w surowicy, widzimy że przy pomiarach zbliżonych zeru pacjenci mieli dużo większą szansę na przeżycie. Przy wyższych wynikach różnica zanika, a dla pomiarów ponad 6tyś jednostek wszyscy pacjenci zmarli.


In [ ]:
import plotly.express as px

fig = px.histogram(dataset, x="serum_creatinine",
                   color="DEATH_EVENT", marginal="rug")
fig.show()

Przy wykresie biorącym pod uwagę czas, możemy zauważyć że większość pacjentów zmarło podczas pierwszych 100 dni leczenia.

In [ ]:
fig = px.histogram(dataset, x="time",
                   color="DEATH_EVENT", marginal="rug")
fig.show()

Przy bardzo niskiej frakcji wyrzutowej pacjenci byli najbardziej narażeni na śmierć, przy wyższych wynikach śmiertelne przypadki stanowią już mały ułamek.

In [ ]:
fig = px.histogram(dataset, x="ejection_fraction",
                   color="DEATH_EVENT", marginal="rug")
fig.show()

Śmiertelność biorąc pod uwagę wiek i analizując poniższy wykres, jak można było się spodziewać, wzrasta wraz z ilością lat pacjenta. Powyżej 70lat wynosi ponad 50 punktów procentowych.

In [ ]:
fig = px.histogram(dataset, x="age",
                   color="DEATH_EVENT", marginal="rug")
fig.show()

Teraz gdy już zapoznaliśmy się bardzien z naszym zbiorem danych możemy przejść do podziału na zbiór treningowy oraz testowy. Niestety nie posiadamy zbyt dużo danych, podzielimy zbiór 80:20 aby nasz model miał wystarczającą ilość informacji do treningu.

In [ ]:
from sklearn.model_selection import train_test_split

features = ['time', 'serum_creatinine', 'ejection_fraction']
x = dataset[features]
y = dataset['DEATH_EVENT']

train_x,test_x,train_y,test_y = train_test_split(x,y, test_size=0.2, random_state=30)

Na początek wypróbujmy model korzystający z regresji logistycznej. Domyślnie korzysta ona z normy L2. Parametr `solver` ustawimy na 'liblinear' ponieważ nasz zbiór jest mały a problem nie jest bardzo złożony. 

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

model_lr = LogisticRegression(solver='liblinear')
model_lr.fit(train_x,train_y)
lr_predict =  model_lr.predict(test_x)
lr_accuracy = accuracy_score(test_y, lr_predict)
print('Accuracy for model using  Linear Regression: ', lr_accuracy*100 , "%")

Warto przyjrzeć się tablicy pomyłek dla naszego naszego modelu. Jak widzimy poniżej nasz model radzi sobie nienajgorze. Na 60 przypadków 12 razy zdiagnozował że serce nie powinno stanąć, a w rzeczywistości było odwrotnie. Z racji że problem rozgrywa się o ludzkie życie lepiej było by czasami fałszywie zaalarmować o możliwości śmierci niż zapewnić że wszystko powinno być w porządku. Potrzebna nam większa czułość. W tym przypadku aby mierzyć poprawność naszego modelu możemy korzystać również z miary F-Beta która pozwala nam regulować balans miedzy precyzja a czułością, parametr Beta to waga czułości. Jak możemy zauważyć gdy traktujemy czułość jako 2 razy ważniejszą od precyzji nasz wynik jest bliski 0.6 co już nie jest tak satysfakcjonującym wynikiem jak powyższy.

In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import fbeta_score

plt.figure()
plot_confusion_matrix(model_lr,test_x, test_y)  
plt.title("Confusion Matrix - Linear Regression Model")
plt.xticks(range(2), ["Heart Not Failed","Heart Fail"], fontsize=16)
plt.yticks(range(2), ["Heart Not Failed","Heart Fail"], fontsize=16)
plt.show()

print('F-Beta score for beta = 2 is ', fbeta_score(test_y, lr_predict, beta=2)*100, '%')

Zmniejszenie siły regularyzacji poprawiłoby pośrednio naszą czułość ale model prawdopodobnie uległby przeuczeniu, dlatego też póki co spróbujemy wykorzystać inny model,mianowicie K-najbliższych sąsiadów, domyślnie k=5. Coś co od razu rzuca się w oczy to, to że trafność obu modeli jest równa (80%) natomiast miara F-Beta dla Beta=2 uległa poprawie. Model w tym przypadku jest bardziej czuły, co w naszym problemie jest bardzo istotne. Co ciekawe, zauważyłem że przy zmniejszeniu liczby sąsiadów miara F-Beta wzrasta jeszcze bardziej, ale jest to spowodowane, podobnie jak w przypadku zmiany siły regularyzacji, przeuczeniem modelu.

In [ ]:
from sklearn.neighbors import KNeighborsClassifier

model_kn = KNeighborsClassifier(n_neighbors=5)
model_kn.fit(train_x,train_y)
kn_predict =  model_kn.predict(test_x)
kn_fbeta = fbeta_score(test_y, kn_predict, beta=2)
kn_accuracy = accuracy_score(test_y, lr_predict)
print('Accuracy for model using K-Neighbors Classifier: ', kn_accuracy*100 , "%")
print('F-Beta score for beta = 2 is ', kn_fbeta*100, '%')

plt.figure()
plot_confusion_matrix(model_kn,test_x, test_y)  
plt.title("Confusion Matrix - K-Neighbors Classifier")
plt.xticks(range(2), ["Heart Not Failed","Heart Fail"], fontsize=16)
plt.yticks(range(2), ["Heart Not Failed","Heart Fail"], fontsize=16)
plt.show()

Spróbujmy w takim razie prosto zoptymalizować algorytm pod kątem miary F2. Model nie jest bardzo złożony więc jedynymi parametrami do optymalizacji w naszym algorytmie będą liczba sąsiadów (minimalnie 3, dla 1 model byłby na pewno przeuczony), oraz p czyli wybór metryki. 

In [ ]:
from optuna.samplers import TPESampler
import optuna
from sklearn.metrics import f1_score
from sklearn.metrics import precision_score
from sklearn.metrics import recall_score

def create_model(trial):
    n_neighbors = trial.suggest_int("n_neighbors", 3, 15)
    p = trial.suggest_int("p", 1, 2)
    
    model = KNeighborsClassifier(
        n_neighbors=n_neighbors, 
        p=p,
    )
    return model

sampler = TPESampler(seed=30)

def objective(trial):
    model = create_model(trial)
    model.fit(train_x, train_y)
    preds = model.predict(test_x)
    return fbeta_score(test_y, preds, beta=2)

study = optuna.create_study(direction="maximize", sampler=sampler)
study.optimize(objective, n_trials=200)

kn_params = study.best_params
kn = KNeighborsClassifier(**kn_params)
kn.fit(train_x, train_y)
preds = kn.predict(test_x)


print('Optimized KNeighborsClassifier accuracy: ', accuracy_score(test_y, preds))
print('Optimized KNeighborsClassifier f1-score', f1_score(test_y, preds))
print('Optimized KNeighborsClassifier f2-score', fbeta_score(test_y, preds,beta=2))
print('Optimized KNeighborsClassifier precision', precision_score(test_y, preds))
print('Optimized KNeighborsClassifier recall', recall_score(test_y, preds))

Po optymalizacji otrzymaliśmy model którego wynik miary F2 wynosi 70% a trafność 85%, czyli poprawiliśmy oba wyniki. Sprawdźmy tablicę pomyłek naszego modelu.

In [ ]:
plt.figure()
plot_confusion_matrix(kn,test_x, test_y)  
plt.title("Confusion Matrix - K-Neighbors Classifier")
plt.xticks(range(2), ["Heart Not Failed","Heart Fail"], fontsize=16)
plt.yticks(range(2), ["Heart Not Failed","Heart Fail"], fontsize=16)
plt.show()

Z racji małego zbioru danych zmiana nie jest bardzo widoczna, ale to wciąż jedno potencjalnie uratowane życie więcej, więc powinniśmy być zadowoloni z poprawy naszego wyniku. 

# Podsumowanie

Najlepszym modelem z przeze mnie wybranych okazał się model korzystający z algorytmu KNeighborsClassifier (K-Najbliższych sąsiadów). Po optymalizacji udało się mu uzyskać wynik 
* `85%` trafności (accuracy score)
* `75%` miary f1
* `70%` miary fbeta przy beta=2

Nie są to może wyniki klasy światowej, ale jestem zadowolony że udało mi się zmaksymalizować możliwości modelu na miarę moich umiejętności. Dużo łatwiej pracowałoby się na zbiorze z większą ilością danych ale nie zawsze jest to możliwe. Patrząc na inne notebooki większość z nich przykładała dużą uwagę do accuracy co w przypadku naszego problemu nie ma tak dużego znaczenia jak miara f2, dlatego też postanowiłem na maksymalizacje tej miary.